# Notebook detailing all data transformation for Power BI :)

## 1) Linking all country sub dimensions based on country numeric code

In [ ]:
-- Setting up git incase I run out of credits :( 
CREATE OR REPLACE API INTEGRATION my_git_api_integration
  API_PROVIDER = git_https_api
  API_ALLOWED_PREFIXES = ('https://github.com/Jcan156/KIWI_notebook.git')
  ENABLED = TRUE;


### Crime Index (subdim_Crime)

In [ ]:
CREATE OR REPLACE TABLE subdim_Crime AS
SELECT
    CR.Country,
    C.country_id,
    CRIMEINDEXVIANUMBEO_2025 AS Crime_index,
    SAFETYINDEXVIANUMBEO_2025 AS Safety_index
FROM dim_Crime AS CR
LEFT JOIN dim_Country C
ON CR.Country = C.Country_Name;

SELECT Country, country_id FROM subdim_Crime WHERE country_id IS NULL           

Seems to be a couple countries without a corresponding code. In most of these cases, its due to the country labeled as "Republic of _" or their new updated name. I hardcoded these values in which is not best practice. 

In [ ]:
UPDATE subdim_Crime
SET country_id = CASE
    WHEN country = 'Moldova' then 498
    WHEN country = 'North Macedonia' then 807
    WHEN country = 'Iran' then 364
    WHEN country = 'Palestine' then 275
    WHEN country = 'Czechia' then 203
    WHEN country = 'United States Virgin Islands' then 850
    WHEN country = 'Syria' then 760
    WHEN country = 'Tanzania' then 834
    WHEN country = 'North Korea' then 408
    ELSE country_id
END;

ALTER TABLE subdim_CRIME
DROP COLUMN country;

SELECT *
FROM subdim_Crime 
WHERE country_id IS NULL;



### Ethnic Group, Race and Religion (subdim_EG_RA_RE)

In [ ]:
-- Same thing for subdim_EG_RA_RE 
CREATE OR REPLACE TABLE subdim_EG_RA_RE AS
SELECT
    ERR.Country,
    C.country_id,
    Ethnic_Group,
    Race,
    Religion
FROM dim_EG_RA_RE AS ERR
LEFT JOIN dim_Country C
ON ERR.Country = C.Country_Name;

SELECT Country, country_id FROM subdim_EG_RA_RE WHERE country_id IS NULL    

Okay already tired of hardcoding so I created a new subdim_Alias table which stores any alias for next time. 

In [ ]:
CREATE OR REPLACE TABLE subdim_Alias (
    alias_id INT AUTOINCREMENT PRIMARY KEY,
    country_id INT,
    alias VARCHAR(255)
);

TRUNCATE TABLE subdim_Alias;

INSERT INTO subdim_Alias (country_id, alias)
VALUES
    (498, 'Moldova'),
    (807, 'North Macedonia'),
    (364, 'Iran'),
    (275, 'Palestine'),
    (203, 'Czechia'),
    (850, 'United States Virgin Islands'),
    (760, 'Syria'),
    (834, 'Tanzania'),
    (408, 'North Korea'),
    (840, 'United States of America');

In [ ]:
UPDATE subdim_EG_RA_RE
SET country_id = a.country_id
FROM subdim_Alias a
WHERE subdim_EG_RA_RE.country_id IS NULL
  AND subdim_EG_RA_RE.country = a.alias;

ALTER TABLE subdim_EG_RA_RE
DROP COLUMN country;

SELECT *
FROM subdim_EG_RA_RE 
WHERE country_id IS NULL;

### Happiness Index (subdim_Happiness)

In [ ]:
-- subdim_Happiness
CREATE OR REPLACE TABLE subdim_Happiness AS
SELECT
    H.Country,
    COALESCE(C1.country_id, C2.country_id) AS country_id,
    H.YEAR,
    H.Rank,
    H.Life_evaluation
FROM dim_Happiness AS H
LEFT JOIN dim_Country AS C1
    ON H.Country = C1.Country_Name
LEFT JOIN subdim_alias AS C2
    ON H.Country = C2.alias
      AND C1.country_id IS NULL;

SELECT Country, country_id FROM subdim_Happiness WHERE country_id IS NULL    

In [ ]:
INSERT INTO subdim_Alias (country_id, alias)
VALUES
    (384, 'Côte d’Ivoire'),
    (180, 'DR Congo'),
    (344, 'Hong Kong SAR of China'),
    (418, 'Lao PDR'),
    (748, 'Eswatini'),
    (410, 'Republic of Korea'),
    (498, 'Republic of Moldova'),
    (275, 'State of Palestine'),
    (158, 'Taiwan Province of China'),
    (792, 'Türkiye');

In [ ]:
-- North Cyprus and Somaliland Region not getting id UN thinks there not a thing (dont cancel me please)
DELETE FROM subdim_Happiness WHERE country = 'Somaliland Region' OR country = 'North Cyprus';

UPDATE subdim_Happiness
SET country_id = a.country_id
FROM subdim_alias a
WHERE subdim_Happiness.country_id IS NULL
  AND subdim_Happiness.country = a.alias;

ALTER TABLE subdim_Happiness
DROP COLUMN country;

SELECT *
FROM subdim_Happiness 
WHERE country_id IS NULL;

In [ ]:
CREATE OR REPLACE TABLE subdim_Happiness AS
WITH latest_year AS (
    SELECT
        country_id,
        life_evaluation,
        rank
    FROM subdim_Happiness
    WHERE year = 2024
)
SELECT *
FROM latest_year;


SELECT * FROM subdim_Happiness LIMIT 5;

### Height Average (subdim_Height)

In [ ]:
-- subdim_Height
CREATE OR REPLACE TABLE subdim_Height AS
SELECT
    He.Country,
    COALESCE(C1.country_id, C2.country_id) AS country_id,
    He.AVERAGEHEIGHTBOYSAGE19,
    He.AVERAGEHEIGHTBOYSAGE19FT,
    He.AVERAGEHEIGHTGIRLSAGE19,
    He.AVERAGEHEIGHTGIRLSAGE19FT,    
FROM dim_Height AS He
LEFT JOIN dim_Country AS C1
    ON He.Country = C1.Country_Name
LEFT JOIN subdim_alias AS C2
    ON He.Country = C2.alias
      AND C1.country_id IS NULL;
      
SELECT Country, country_id FROM subdim_Height WHERE country_id IS NULL    

In [ ]:
INSERT INTO subdim_Alias (country_id, alias)
VALUES
    (418, 'Laos'),
    (583, 'Micronesia'),
    (180, 'Republic of the Congo');
    

In [ ]:
DELETE FROM subdim_height WHERE country = 'Global Average';

UPDATE subdim_height
SET country_id = a.country_id
FROM subdim_alias a
WHERE subdim_height.country_id IS NULL
  AND subdim_height.country = a.alias;

ALTER TABLE subdim_height
DROP COLUMN country;

SELECT *
FROM subdim_height 
WHERE country_id IS NULL;

### LGBTQ Acceptance (subdim_LGBTQ_A)

In [ ]:
-- subdim_LGBTQ_A
CREATE OR REPLACE TABLE subdim_LGBTQ_A AS
SELECT
    LA.Country,
    COALESCE(C1.country_id, C2.country_id) AS country_id,
    LA.GAI,
    LA.RANK
FROM dim_LGBTQ_A AS LA
LEFT JOIN dim_Country AS C1
    ON LA.Country = C1.Country_Name
LEFT JOIN subdim_alias AS C2
    ON LA.Country = C2.alias
      AND C1.country_id IS NULL;
      
SELECT Country, country_id FROM subdim_LGBTQ_A WHERE country_id IS NULL    

In [ ]:
INSERT INTO subdim_Alias (country_id, alias)
VALUES
    (384, 'Cote d’Ivoire'),
    (180, 'Democratic Republic of the Congo'),
    (826, 'Great Britain'),
    (446, 'Macau SAR'),
    (70, 'Bosnia Herzegovina');
    

In [ ]:
-- Nagorno-Karabakh is part of Azerbajin (According to UN chill)
DELETE FROM subdim_LGBTQ_A WHERE country = 'Somaliland' OR country = 'Northern Cyprus' OR country = 'Nagorno-Karabakh';

UPDATE subdim_LGBTQ_A
SET country_id = a.country_id
FROM subdim_alias a
WHERE subdim_LGBTQ_A.country_id IS NULL
  AND subdim_LGBTQ_A.country = a.alias;

ALTER TABLE subdim_LGBTQ_A
DROP COLUMN country;

SELECT *
FROM subdim_LGBTQ_A 
WHERE country_id IS NULL;

### LGBTQ Population Percentage (subdim_LGBTQ_P)

In [ ]:
-- subdim_LGBTQ_P
CREATE OR REPLACE TABLE subdim_LGBTQ_P AS
SELECT
    LP.Country,
    COALESCE(C1.country_id, C2.country_id) AS country_id,
    LP.LGBTQPOPULATIONPCTOFTOTAL_2025
FROM dim_LGBTQ_P AS LP
LEFT JOIN dim_Country AS C1
    ON LP.Country = C1.Country_Name
LEFT JOIN subdim_alias AS C2
    ON LP.Country = C2.alias
      AND C1.country_id IS NULL;

ALTER TABLE subdim_LGBTQ_P
DROP COLUMN country;
      
SELECT * FROM subdim_LGBTQ_P WHERE country_id IS NULL  

### Obesity % in population (subdim_Obesity)

In [ ]:
-- subdim_Obesity
CREATE OR REPLACE TABLE subdim_Obesity AS
SELECT
    O.Country,
    COALESCE(C1.country_id, C2.country_id) AS country_id,
    O.FEMALEOBESITYRATEWHO_2022,
    O.MALEOBESITYRATEWHO_2022,
    O.TOTALOBESITYRATEWHO_2022
FROM dim_Obesity AS O
LEFT JOIN dim_Country AS C1
    ON O.Country = C1.Country_Name
LEFT JOIN subdim_alias AS C2
    ON O.Country = C2.alias
      AND C1.country_id IS NULL;

ALTER TABLE subdim_Obesity
DROP COLUMN country;
      
SELECT * FROM subdim_Obesity WHERE country_id IS NULL    

### Population (subdim_Population)
The population dataset includes the value for multiple regions such as continent and major regions. We are only intrested in the value for countries and there most recent value.

In [ ]:
-- First filter latest year and the series is "Population mid-year estimates (millions)"
CREATE OR REPLACE TABLE dim_Population_temp AS
WITH latest_year_pop AS (
    SELECT
            country,
            value_millions,
            value_4real,
            year,
            ROW_NUMBER() OVER (
                PARTITION BY country
                ORDER BY year DESC
            ) AS rn
    FROM dim_Population
    WHERE series = 'Population mid-year estimates (millions)'
)
SELECT
    country,
    value_millions,
    value_4real
FROM latest_year_pop
WHERE rn = 1;

SELECT * FROM dim_Population_Temp LIMIT 5

In [ ]:
-- -- Same as before, this time left joining on country table to get all the countries and leave out any extra population meassures
-- CREATE OR REPLACE TABLE subdim_Population AS
-- SELECT
--     C.country_name,
--     C.country_id,
--     COALESCE(p1.value_millions, p2.value_millions) AS value_millions,
--     COALESCE(p1.value_4real,    p2.value_4real) AS value_4real
-- FROM dim_Country C
-- LEFT JOIN dim_Population_Temp p1
-- ON C.country_name = p1.country
-- LEFT JOIN subdim_Alias a
-- ON c.country_id = a.country_id
-- LEFT JOIN dim_Population_Temp p2
-- ON a.alias = p2.country;

-- SELECT country_name FROM subdim_Population WHERE value_millions IS NULL

CREATE OR REPLACE TABLE subdim_Population AS
SELECT
    P.country,
    COALESCE(C1.country_id, C2.country_id) AS country_id,
    P.value_millions,
    P.value_4real
FROM dim_Population_Temp AS P
LEFT JOIN dim_Country AS C1
    ON P.country = C1.country_name
LEFT JOIN subdim_Alias AS C2
    ON P.country = C2.alias
   AND C1.country_id IS NULL;
   
SELECT country FROM subdim_Population WHERE country_id IS NULL
